In [2]:
from itertools import islice
with open("wikicorpus_01") as myfile:
    head = list(islice(myfile, 651))

In [3]:
print(head[:9])

['Néstor_Gabriel_Martinena néstor_gabriel_martinena NP00000 0\n', '( ( Fpa 0\n', 'La el DA0FS0 0\n', 'Plata plata NP00000 10501473\n', ', , Fc 0\n', 'Buenos_Aires buenos_aires NP00000 0\n', ', , Fc 0\n', 'Argentina argentina NP00000 02035234\n', ', , Fc 0\n']


In [4]:
sent = []
for s in head:
    sent.append(s.split())

In [5]:
sent[:5]

[['Néstor_Gabriel_Martinena', 'néstor_gabriel_martinena', 'NP00000', '0'],
 ['(', '(', 'Fpa', '0'],
 ['La', 'el', 'DA0FS0', '0'],
 ['Plata', 'plata', 'NP00000', '10501473'],
 [',', ',', 'Fc', '0']]

In [6]:
words = []
sentences = []
for s in sent:
    if len(s) == 4 and s[0] not in {'(', ')', ','} and '[' not in s[1]:
        words.append((s[1], s[2], s[3]))
        #if 'endofarticle' in s[1]:
        #    sentences.append(words)
        #    words = []

In [7]:
count = dict()
for fst,snd,trd in words:
    if fst not in count:
        count[fst] = 1
    else:
        count[fst] += 1

In [8]:
count['entrar']

2

In [9]:
words[:4]

[('néstor_gabriel_martinena', 'NP00000', '0'),
 ('el', 'DA0FS0', '0'),
 ('plata', 'NP00000', '10501473'),
 ('buenos_aires', 'NP00000', '0')]

In [10]:
aux = []
check = []
for fst,snd,trd in words:
    aux.append((fst,snd, trd))
    if fst == '.':
        check.append(aux)
        aux = []

In [11]:
check[:3]

[[('néstor_gabriel_martinena', 'NP00000', '0'),
  ('el', 'DA0FS0', '0'),
  ('plata', 'NP00000', '10501473'),
  ('buenos_aires', 'NP00000', '0'),
  ('argentina', 'NP00000', '02035234'),
  ('ser', 'VSIP3S0', '01775973'),
  ('uno', 'DI0MS0', '0'),
  ('futbolista', 'NCCS000', '0'),
  ('argentino', 'AQ0MS0', '00278090'),
  ('.', 'Fp', '0')],
 [('jugar', 'VMM02S0', '00727813'),
  ('de', 'SPS00', '0'),
  ('delantero', 'NCMS000', '00466114'),
  ('y', 'CC', '0'),
  ('su', 'DP3CS0', '0'),
  ('equipo', 'NCMS000', '06093198'),
  ('actual', 'AQ0CS0', '01667781'),
  ('ser', 'VSIP3S0', '01775973'),
  ('gimnasia', 'NP00000', '00275488'),
  ('y', 'CC', '0'),
  ('esgrima_la_plata_de_la_primera_división_de_argentina', 'NP00000', '0'),
  ('.', 'Fp', '0')],
 [('trayectoria', 'NP00000', '06709272'), ('.', 'Fp', '0')]]

In [12]:
from collections import defaultdict

In [13]:
dicc_backward = defaultdict(list)
dicc_forward = defaultdict(list)
for s in check:
    for i,w in enumerate(s):
        if i != 0:
            dicc_backward[w[0]].append((s[i-1][0], s[i-1][1]))
        else:
            dicc_backward[w[0]].append(('[W.Start]', '[T.Start]'))
        if i != len(s)-1:
            dicc_forward[w[0]].append((s[i+1][0], s[i+1][1]))
        else:
            dicc_forward[w[0]].append(('[W.End]', '[T.End]'))

In [14]:
dicc_backward['entrar']

[('river_plate', 'NP00000'), ('el', 'DA0MS0')]

In [15]:
dicc_forward['entrar']

[('desde', 'SPS00'), ('a', 'SPS00')]

In [17]:
dicc = dict()
vec = dict()
for fst,snd,trd in words:
    features = defaultdict(int)
    pos = 'PoS__' + snd
    if fst not in dicc and fst not in {'.', ',', ':', ';'} and fst.isalpha() and count[fst] > 2:
        dicc[fst] = features
        vec[fst] = trd
        features[pos] = 1
        for i in dicc_backward[fst]:
            features[i[0]+"-1"] += 1
            features[i[1]+"-1"] += 1
        for i in dicc_forward[fst]:
            features[i[0]+"+1"] += 1
            features[i[0]+"+1"] += 1
        
    elif fst in dicc:
        has_it = dicc[fst]
        if pos in has_it:
            has_it[pos] += 1
        else:
            has_it[pos] = 1

In [18]:
dicc['con']

defaultdict(int,
            {'35+1': 2,
             'AQ0FP0-1': 1,
             'NCFS000-1': 1,
             'NCMP000-1': 1,
             'NCMS000-1': 3,
             'NP00000-1': 1,
             'PoS__SPS00': 9,
             'VMII1S0-1': 1,
             'VMIP3S0-1': 1,
             'arco+1': 2,
             'balcón+1': 2,
             'balcón-1': 1,
             'bilbao-1': 1,
             'conectar-1': 1,
             'contar-1': 1,
             'cubo+1': 2,
             'decoración+1': 2,
             'desarrollo+1': 2,
             'el+1': 2,
             'escudo-1': 1,
             'etapa-1': 1,
             'gótico-1': 1,
             'realengo-1': 1,
             'recinto-1': 1,
             'regidor+1': 2,
             'trespaderne+1': 2})

In [19]:
vec

{'a': '0',
 'altura': '03998008',
 'con': '0',
 'cuyo': '0',
 'de': '0',
 'el': '0',
 'en': '0',
 'equipo': '06093198',
 'localidad': '06368526',
 'minuto': '10943650',
 'partido': '05554348',
 'planta': '03138429',
 'por': '0',
 'que': '0',
 'ser': '01775973',
 'situar': '01354457',
 'su': '0',
 'tiempo': '10850147',
 'uno': '0',
 'y': '0'}

In [20]:
matrix = []
vector = []
final_check = []
for key, value in dicc.items():    #preparo la lista de dicts y la lista de chequeo para recuperar el nombre al final
    final_check.append(key)
    matrix.append(value)
for key, value in vec.items():       #vector de synsets
    vector.append(int(value))

In [21]:
vector

[0,
 1775973,
 0,
 0,
 0,
 0,
 6093198,
 0,
 0,
 10943650,
 10850147,
 5554348,
 0,
 0,
 6368526,
 1354457,
 0,
 3998008,
 0,
 3138429]

In [22]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer()
X = v.fit_transform(matrix)

In [23]:
X

<20x325 sparse matrix of type '<class 'numpy.float64'>'
	with 450 stored elements in Compressed Sparse Row format>

In [24]:
##########################################Supervisada, clase: synset ##################################################

In [25]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [26]:
Y = SelectKBest(chi2, k=4).fit_transform(X, vector)

In [27]:
#Clustering
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3)
km.fit(Y)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [28]:
labels = km.predict(Y)